# Chronos Bolt Base - Zero Shot, With covariates

Document: https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-chronos.html#incorporating-the-covariates

**The predictions of the covariate regressor are subtracted from the target column, and the univariate model then forecasts the residuals.**

In [1]:
import datetime

import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from sklearn.metrics import root_mean_squared_error

In [2]:
# Experiment設定
EXPERIMENT_NAME = 'ChronosBolt_0Shot_Covariate'
MODEL = "bolt_base"
FORECAST_HORIZON = 6

In [6]:
df_train = pd.read_csv('../data/walmart_ts_6_fcst_grp_train.csv')
df_test = pd.read_csv('../data/walmart_ts_6_fcst_grp_test.csv')
print(df_train.shape, df_test.shape)
#df_train.head()

# 日付カラムを型変換する場合
#df_train['Date'] = pd.to_datetime(df_train['Date'])
#df_test['Date'] = pd.to_datetime(df_test['Date'])

# 単一のグループIDを作成
df_train['Store_Dept'] = [str(i)+'_'+str(j) for i,j in zip(df_train['Store'], df_train['Dept'])]
df_test['Store_Dept'] = [str(i)+'_'+str(j) for i,j in zip(df_test['Store'], df_test['Dept'])]

# 利用カラムのみを選択
df_train = df_train[['Store_Dept','Date','Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday']]
df_test = df_test[['Store_Dept','Date','Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday']]
print(df_train.shape, df_test.shape)
df_train.head()

(702, 11) (36, 11)
(702, 9) (36, 9)


,Store_Dept,Date,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,IsHoliday
0,4_4,2010/2/5,59554.57,-1.0,-1.0,-1.0,-1.0,-1.0,0
1,4_4,2010/2/12,54069.82,-1.0,-1.0,-1.0,-1.0,-1.0,1
2,4_4,2010/2/19,53939.17,-1.0,-1.0,-1.0,-1.0,-1.0,0
3,4_4,2010/2/26,54687.08,-1.0,-1.0,-1.0,-1.0,-1.0,0
4,4_4,2010/3/5,56959.02,-1.0,-1.0,-1.0,-1.0,-1.0,0


In [7]:
# TimeSeriesDataFrameへ変換
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    id_column='Store_Dept',
    timestamp_column="Date"
)
print(type(train_data))
train_data.head()

<class 'autogluon.timeseries.dataset.ts_dataframe.TimeSeriesDataFrame'>


Weekly_Sales  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
item_id timestamp                                                              
4_4     2010-02-05      59554.57       -1.0       -1.0       -1.0       -1.0   
        2010-02-12      54069.82       -1.0       -1.0       -1.0       -1.0   
        2010-02-19      53939.17       -1.0       -1.0       -1.0       -1.0   
        2010-02-26      54687.08       -1.0       -1.0       -1.0       -1.0   
        2010-03-05      56959.02       -1.0       -1.0       -1.0       -1.0   

                    MarkDown5  IsHoliday  
item_id timestamp                         
4_4     2010-02-05       -1.0          0  
        2010-02-12       -1.0          1  
        2010-02-19       -1.0          0  
        2010-02-26       -1.0          0  
        2010-03-05       -1.0          0

In [10]:
# Test Dataも同様に変換。Covariatesを利用する場合、予測時に必要
test_data = TimeSeriesDataFrame.from_data_frame(
    df_test,
    #id_column=['Store', 'Dept'],
    id_column='Store_Dept',
    timestamp_column="Date"
)
test_data.head()

Weekly_Sales  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
item_id timestamp                                                              
4_4     2012-05-04      60576.41   13737.91       -1.0      93.65    6993.97   
        2012-05-11      57583.94   20499.88       86.0     103.05    5005.96   
        2012-05-18      60832.91    6531.12       -1.0      99.91    2290.18   
        2012-05-25      57781.87    4330.02      436.0     862.31    3268.41   
        2012-06-01      62758.52   10165.22        8.6      47.93    2676.33   

                    MarkDown5  IsHoliday  
item_id timestamp                         
4_4     2012-05-04    4541.89          0  
        2012-05-11    3815.16          0  
        2012-05-18    2521.84          0  
        2012-05-25    3749.53          0  
        2012-06-01    4896.34          0

In [8]:
%%time

log_dir = EXPERIMENT_NAME +'_'+ datetime.datetime.now().strftime('%Y-%m-%d_%Hh%Mm%Ss')
print(log_dir)

predictor = TimeSeriesPredictor(
    target="Weekly_Sales",
    eval_metric="RMSE",
    prediction_length=FORECAST_HORIZON,
    path="./tmp/"+log_dir,
    
    known_covariates_names=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday'],
).fit(
    train_data,
    hyperparameters={
        "Chronos": [
            {
                "model_path": MODEL,
                #"covariate_regressor": "XGB",  # XGBoost  -> カーネルが落ちてしまう、、。
                "covariate_regressor": "CAT",  # CatBoost
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
        ],
    },
    enable_ensemble=False,
    time_limit=300,
)

Beginning AutoGluon training... Time limit = 300s
AutoGluon will save models to '/Users/YShimada-MBP16/Documents/Code_Github/DL_for_ImageData_and_Finetuning/TimeSeries_WM_simple/code/tmp/ChronosBolt_0Shot_Covariate_2025-03-01_09h22m16s'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.14
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:22:00 PST 2025; root:xnu-11215.81.4~3/RELEASE_X86_64
CPU Count:          16
GPU Count:          0
Memory Avail:       14.58 GB / 32.00 GB (45.6%)
Disk Space Avail:   1433.36 GB / 1863.39 GB (76.9%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': RMSE,
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'WithRegressor'},
                                  'covariate_regressor': 'CAT',
                                  'model_path': 'bolt_base',
                                  'target_scaler': 'standard'}]},

ChronosBolt_0Shot_Covariate_2025-03-01_09h22m16s


	-2431.9285    = Validation score (-RMSE)
	5.15    s     = Training runtime
	2.06    s     = Validation (prediction) runtime
Training complete. Models trained: ['ChronosWithRegressor[bolt_base]']
Total runtime: 7.22 s
Best model: ChronosWithRegressor[bolt_base]
Best model score: -2431.9285


CPU times: user 6.57 s, sys: 2.12 s, total: 8.69 s
Wall time: 7.31 s


In [11]:
# 与えられたデータの最後の日付から、prediction_length分を予測
predictions = predictor.predict(train_data, known_covariates=test_data)
predictions

Model not specified in predict, will default to the model with the best validation score: ChronosWithRegressor[bolt_base]


mean           0.1           0.2           0.3  \
item_id timestamp                                                            
4_4     2012-05-04  58062.427161  54682.150688  55719.936149  56521.880196   
        2012-05-11  58294.475125  54774.458346  55809.417640  56657.726292   
        2012-05-18  56786.960989  53222.402726  54270.630099  55140.561540   
        2012-05-25  62692.049160  59088.478639  60170.472286  61047.612732   
        2012-06-01  58436.575058  54632.498086  55785.887569  56709.269480   
        2012-06-08  59742.964675  55956.965790  57111.393334  58027.399544   
8_13    2012-05-04  23948.211814  22456.628968  22957.749272  23324.866758   
        2012-05-11  24227.802958  22639.257052  23158.018130  23552.159044   
        2012-05-18  24775.855029  23156.964381  23688.349834  24090.175327   
        2012-05-25  25232.293831  23601.236697  24144.071359  24549.542091   
        2012-06-01  24075.718955  22419.200304  22971.254999  23383.118923   
        2012-06-08  25213.337391  23562.983383  24114.983929  24524.282115   
1_13    2012-05-04  41291.752900  38814.186230  39678.129767  40269.080520   
        2012-05-11  40584.783436  37956.065965  38842.659133  39483.987328   
        2012-05-18  39836.332801  37211.360169  38106.072337  38757.651548   
        2012-05-25  41319.168581  38568.421552  39509.405419  40179.993730   
        2012-06-01  40387.143027  37328.359995  38361.664831  39108.757611   
        2012-06-08  41883.470149  38738.677012  39826.068841  40579.565478   
13_8    2012-05-04  34887.062762  32740.004170  33455.277567  33986.566797   
        2012-05-11  34662.268410  32434.971059  33167.530513  33722.637538   
        2012-05-18  34154.068941  31919.232733  32666.943874  33227.884251   
        2012-05-25  35142.906196  32891.577825  33654.549390  34218.021422   
        2012-06-01  34736.211058  32450.990323  33226.663618  33799.231297   
        2012-06-08  35879.664618  33587.348519  34366.559139  34942.774618   
19_8    2012-05-04  42995.031434  40478.524989  41321.172956  41942.098560   
        2012-05-11  41444.873862  38802.024955  39663.613316  40319.515675   
        2012-05-18  41546.200379  38893.800128  39765.488437  40425.595149   
        2012-05-25  44346.070933  41678.260637  42565.495478  43224.709374   
        2012-06-01  40902.728476  38196.050476  39096.535783  39765.787122   
        2012-06-08  42704.600351  39968.103400  40882.829621  41559.380785   
31_13   2012-05-04  43009.571418  40749.173492  41536.528470  42095.498520   
        2012-05-11  42988.379565  40500.247308  41340.834105  41960.907963   
        2012-05-18  44191.414783  41613.515096  42491.962994  43135.573950   
        2012-05-25  44149.611862  41556.949543  42451.297721  43098.186983   
        2012-06-01  42732.508849  40074.412300  40997.343932  41662.652039   
        2012-06-08  43484.634128  40779.047034  41719.994180  42396.221260   

                             0.4           0.5           0.6           0.7  \
item_id timestamp                                                            
4_4     2012-05-04  57281.437911  58062.427161  58907.675418  59859.390265   
        2012-05-11  57461.194560  58294.475125  59181.259886  60186.337566   
        2012-05-18  55961.425707  56786.960989  57671.374170  58685.326339   
        2012-05-25  61870.962362  62692.049160  63582.907432  64608.690998   
        2012-06-01  57566.370312  58436.575058  59368.708677  60424.989783   
        2012-06-08  58873.205972  59742.964675  60670.536513  61754.948384   
8_13    2012-05-04  23645.702088  23948.211814  24262.552611  24617.516069   
        2012-05-11  23898.459896  24227.802958  24566.008096  24950.250486   
        2012-05-18  24443.008440  24775.855029  25118.260280  25508.713604   
        2012-05-25  24900.819897  25232.293831  25580.003834  25976.843393   
        2012-06-01  23736.558373  24075.718955  24424.539672  24823.308652   
        2012-06-08  24877.534906  25213.337391  2

In [12]:
df_pred = predictions.to_data_frame().reset_index()

print('RMSE: ', root_mean_squared_error(y_true=df_test['Weekly_Sales'], y_pred=df_pred['mean']))

df_pred.to_csv('../Chronos_results/{}.csv'.format(EXPERIMENT_NAME), index=False)

RMSE:  1956.8658785712619
